In [48]:
import pandas as pd

In [49]:
# set max display rows
pd.set_option("display.max_rows", 1000)
# set max display columns
pd.set_option("display.max_columns", 1000)
# set max display width
pd.set_option("display.width", 1000)

# Load Data

In [50]:
CIL_df = pd.read_csv(
    "results/class_incremental-trials=10-batch_size=128-window_size=50-num_voters=8-width=64-CAI-experiments-feb16-tabledata-test-feb16.csv"
)
DIL_df = pd.read_csv(
    "results/domain_incremental-trials=10-batch_size=128-window_size=400-num_voters=8-width=64-CAI-experiments-feb16-tabledata-test-feb16.csv"
)

In [56]:
CIL_df.head()

,ensemble,avg_test_acc,context_1_test_acc,context_2_test_acc,context_3_test_acc,context_4_test_acc,context_5_test_acc
0,GEM,$95.78 \pm 0.22$,$98.39 \pm 0.23$,$95.19 \pm 0.65$,$94.98 \pm 0.76$,$95.79 \pm 0.35$,$94.55 \pm 0.33$
1,StudentExpert-num_gurus_train-1-num_gurus_test-1,$91.75 \pm 0.39$,$93.05 \pm 0.85$,$89.95 \pm 1.07$,$92.08 \pm 0.42$,$92.90 \pm 0.15$,$90.79 \pm 0.33$
2,StudentExpert-num_gurus_train-1-num_gurus_test-4,$90.77 \pm 1.27$,$89.84 \pm 4.89$,$90.16 \pm 0.68$,$90.10 \pm 2.99$,$92.90 \pm 0.10$,$90.82 \pm 0.29$
3,StudentExpert-num_gurus_train-1-num_gurus_test-3,$90.10 \pm 1.60$,$87.40 \pm 5.40$,$89.06 \pm 1.83$,$90.40 \pm 3.34$,$92.94 \pm 0.13$,$90.73 \pm 0.31$
4,StudentExpert-num_gurus_train-1-num_gurus_test-2,$89.70 \pm 3.43$,$86.42 \pm 11.51$,$86.69 \pm 6.12$,$91.59 \pm 1.04$,$92.98 \pm 0.26$,$90.82 \pm 0.39$


# CIL

In [54]:
# Our best are: StudentExpert-num_gurus_train-1-num_gurus_test-1 and probabilistic_better-balanced_accuracy_score-num_gurus-1
# drop all rows containing num_gurus aside from the rows above StudentExpert-num_gurus_train-1-num_gurus_test-1 and probabilistic_better-balanced_accuracy_score-num_gurus-1
CIL_our_best = CIL_df[
    CIL_df["ensemble"].str.contains("StudentExpert-num_gurus_train-1-num_gurus_test-1")
    | CIL_df["ensemble"].str.contains(
        "probabilistic_better-balanced_accuracy_score-num_gurus-1"
    )
]
# get all rows not containing num_gurus
CIL_other = CIL_df[~CIL_df["ensemble"].str.contains("num_gurus")]

# combine the two dataframes
CIL_table_df = pd.concat([CIL_our_best, CIL_other])
# Sort the dataframe by avg_test_acc
CIL_table_df = CIL_table_df.sort_values(by="avg_test_acc", ascending=False)
# reset the index
CIL_table_df = CIL_table_df.reset_index(drop=True)


# Use replay: GEM, Replay, GDumb, MIR, AGEM, CoPE
# LFL, RWALK do not use task identities
# Drop train generator after exp

# Put Student Expert row first
# then put the methods that use replay: GEM, AGEM, Replay, GDumb, MIR, CoPE
# then put all the other methods

CIL_ordered_test = CIL_table_df[
    CIL_table_df["ensemble"].str.contains(
        "StudentExpert-num_gurus_train-1-num_gurus_test-1"
    )
]
CIL_memory = CIL_table_df[
    CIL_table_df["ensemble"].str.contains("GEM")
    | CIL_table_df["ensemble"].str.contains("AGEM")
    | CIL_table_df["ensemble"].str.contains("Replay")
    | CIL_table_df["ensemble"].str.contains("GDumb")
    | CIL_table_df["ensemble"].str.contains("MIR")
    | CIL_table_df["ensemble"].str.contains("CoPE")
]

# CIL_other will be all methods not contained in CIL_memory or CIL_ordered_test
CIL_other = CIL_table_df[
    ~CIL_table_df["ensemble"].str.contains(
        "StudentExpert-num_gurus_train-1-num_gurus_test-1"
    )
    & ~CIL_table_df["ensemble"].str.contains("GEM")
    & ~CIL_table_df["ensemble"].str.contains("AGEM")
    & ~CIL_table_df["ensemble"].str.contains("Replay")
    & ~CIL_table_df["ensemble"].str.contains("GDumb")
    & ~CIL_table_df["ensemble"].str.contains("MIR")
    & ~CIL_table_df["ensemble"].str.contains("CoPE")
]

# sort all three dataframes by avg_test_acc
CIL_ordered_test = CIL_ordered_test.sort_values(by="avg_test_acc", ascending=False)
CIL_memory = CIL_memory.sort_values(by="avg_test_acc", ascending=False)
CIL_other = CIL_other.sort_values(by="avg_test_acc", ascending=False)

# reset the index
CIL_ordered_test = CIL_ordered_test.reset_index(drop=True)
CIL_memory = CIL_memory.reset_index(drop=True)
CIL_other = CIL_other.reset_index(drop=True)

# combine the three dataframes
CIL_table_df = pd.concat([CIL_ordered_test, CIL_memory, CIL_other])

# Rename the CIL_table_df columns to "Method", "Mean Acc.", "$C_1$", "$C_2$", "$C_3$", "$C_4$", "$C_5$"

CIL_table_df.columns = [
    "Method",
    "Mean Acc.",
    "$C_1$",
    "$C_2$",
    "$C_3$",
    "$C_4$",
    "$C_5$",
]
CIL_table_df

# save the dataframe to a csv file
CIL_table_df.to_csv("results/A_CIL_table.csv", index=False)

# DIL

In [57]:
DIL_df.head(100)

,ensemble,avg_test_acc,context_1_test_acc,context_2_test_acc,context_3_test_acc,context_4_test_acc,context_5_test_acc
0,GEM,$95.41 \pm 0.60$,$94.66 \pm 1.44$,$94.93 \pm 1.50$,$94.72 \pm 0.90$,$95.71 \pm 0.81$,$97.05 \pm 0.21$
1,StudentExpert-num_train_gurus-2-num_test_gurus-2,$92.46 \pm 1.39$,$93.59 \pm 0.62$,$89.21 \pm 3.27$,$92.34 \pm 2.41$,$92.12 \pm 3.94$,$95.05 \pm 0.68$
2,StudentExpert-num_train_gurus-2-num_test_gurus-3,$91.72 \pm 1.31$,$93.56 \pm 0.76$,$87.53 \pm 4.06$,$90.43 \pm 2.10$,$92.16 \pm 3.07$,$94.93 \pm 0.59$
3,StudentExpert-num_train_gurus-2-num_test_gurus-1,$91.27 \pm 3.20$,$93.29 \pm 0.65$,$84.19 \pm 16.93$,$92.25 \pm 2.31$,$91.80 \pm 4.10$,$94.82 \pm 0.64$
4,StudentExpert-num_train_gurus-2-num_test_gurus-4,$89.03 \pm 4.05$,$93.05 \pm 1.42$,$77.91 \pm 19.79$,$88.64 \pm 4.82$,$91.29 \pm 3.18$,$94.26 \pm 0.84$
5,StudentExpert-num_train_gurus-1-num_test_gurus-1,$88.41 \pm 4.91$,$93.42 \pm 0.50$,$82.32 \pm 19.86$,$80.05 \pm 20.61$,$91.71 \pm 2.66$,$94.57 \pm 0.69$
6,StudentExpert-num_train_gurus-1-num_test_gurus-2,$88.21 \pm 4.07$,$93.40 \pm 0.38$,$78.25 \pm 19.36$,$85.12 \pm 13.66$,$90.46 \pm 4.40$,$93.85 \pm 0.68$
7,StudentExpert-num_train_gurus-1-num_test_gurus-3,$87.62 \pm 4.44$,$93.78 \pm 0.53$,$79.18 \pm 17.80$,$82.53 \pm 16.39$,$89.59 \pm 4.36$,$93.01 \pm 1.20$
8,StudentExpert-num_train_gurus-3-num_test_gurus-3,$86.45 \pm 5.69$,$72.12 \pm 26.12$,$82.53 \pm 18.42$,$90.50 \pm 3.45$,$91.85 \pm 4.03$,$95.25 \pm 0.63$
9,StudentExpert-num_train_gurus-3-num_test_gurus-2,$85.88 \pm 5.84$,$73.29 \pm 25.21$,$77.30 \pm 23.82$,$91.08 \pm 3.86$,$92.79 \pm 2.41$,$94.92 \pm 0.66$


In [55]:
# Our best methods: StudentExpert-num_train_gurus-2-num_test_gurus-2, random_better-f1_score-num_gurus-2
# drop all rows containing num_gurus aside from the rows above StudentExpert-num_train_gurus-2-num_test_gurus-2 and random_better-f1_score-num_gurus-2
DIL_our_best = DIL_df[
    DIL_df["ensemble"].str.contains("StudentExpert-num_train_gurus-2-num_test_gurus-2")
    | DIL_df["ensemble"].str.contains("random_better-f1_score-num_gurus-2")
]
# get all rows not containing num_gurus
DIL_other = DIL_df[~DIL_df["ensemble"].str.contains("num_gurus")&~DIL_df["ensemble"].str.contains("StudentExpert")]

# combine the two dataframes
DIL_table_df = pd.concat([DIL_our_best, DIL_other])
# Sort the dataframe by avg_test_acc
DIL_table_df = DIL_table_df.sort_values(by="avg_test_acc", ascending=False)
# reset the index
DIL_table_df = DIL_table_df.reset_index(drop=True)

# Put Student Expert row first
# then put the methods that use replay: GEM, AGEM, Replay, GDumb, MIR, CoPE
# then put all the other methods

DIL_ordered_test = DIL_table_df[
    DIL_table_df["ensemble"].str.contains("StudentExpert-num_train_gurus-2-num_test_gurus-2")
]
DIL_memory = DIL_table_df[
    DIL_table_df["ensemble"].str.contains("GEM")
    | DIL_table_df["ensemble"].str.contains("AGEM")
    | DIL_table_df["ensemble"].str.contains("Replay")
    | DIL_table_df["ensemble"].str.contains("GDumb")
    | DIL_table_df["ensemble"].str.contains("MIR")
    | DIL_table_df["ensemble"].str.contains("CoPE")
]

# DIL_other will be all methods not contained in DIL_memory or DIL_ordered_test
DIL_other = DIL_table_df[
    ~DIL_table_df["ensemble"].str.contains("StudentExpert-num_train_gurus-2-num_test_gurus-2")
    & ~DIL_table_df["ensemble"].str.contains("GEM")
    & ~DIL_table_df["ensemble"].str.contains("AGEM")
    & ~DIL_table_df["ensemble"].str.contains("Replay")
    & ~DIL_table_df["ensemble"].str.contains("GDumb")
    & ~DIL_table_df["ensemble"].str.contains("MIR")
    & ~DIL_table_df["ensemble"].str.contains("CoPE")
]

# sort all three dataframes by avg_test_acc
DIL_ordered_test = DIL_ordered_test.sort_values(by="avg_test_acc", ascending=False)
DIL_memory = DIL_memory.sort_values(by="avg_test_acc", ascending=False)
DIL_other = DIL_other.sort_values(by="avg_test_acc", ascending=False)

# reset the index
DIL_ordered_test = DIL_ordered_test.reset_index(drop=True)
DIL_memory = DIL_memory.reset_index(drop=True)
DIL_other = DIL_other.reset_index(drop=True)

# combine the three dataframes
DIL_table_df = pd.concat([DIL_ordered_test, DIL_memory, DIL_other])

# Rename the DIL_table_df columns to "Method", "Mean Acc.", "$C_1$", "$C_2$", "$C_3$", "$C_4$", "$C_5$"

DIL_table_df.columns = [
    "Method",
    "Mean Acc.",
    "$C_1$",
    "$C_2$",
    "$C_3$",
    "$C_4$",
    "$C_5$",
]

# save the dataframe to a csv file
DIL_table_df.to_csv("results/A_DIL_table.csv", index=False)